# Notebook to be used with ESRIDB connection.

**Script to create MGOPermits Table from the MGOPermits Table on KCC-SQL1**

In [ ]:
SET ANSI_PADDING ON
GO

USE PROD
GO

TRUNCATE Table [gisadmin].[MGOPermits]

Insert into [gisadmin].[MGOPermits] (
    [PARCELID],
    [AppType],
    [PermitStatus],
    [CODate],
    [AppDate],
    [IssueDate],
    [PermitYear],
    [PermitNumber],
    [Designation]
)
select CONCAT([PARCELID], '01'), [ApplicationType], [Status], [COIssuedDate], [Created], '', '', [ProjectNumber], [Designation]
from [KCC-SQL1].[Assess50StagedData_Kent].[dbo].[MGOPermitData]

**Cell to query the AS400 Server.**

In [ ]:
-- SELECT *
--   FROM [PROD].[gisadmin].[All_Permits]

-- select *
-- from [KCLCAS400].[S78114C0].[HTSUSR].[BPK001P2FL]
-- where [YR] = '20' and [BPPYER] = '20'
-- order by [MSPIN]

select count(*)
from [KCLCAS400].[S78114C0].[HTSUSR].[BPK001P2FL]
where [MSPIN] LIKE '%-00001'
-- where [MSPIN] = '4-00-04600-02-0400[KCLCAS400].[S78114C0].[HTSUSR].[BPK001P2FL]-00001' and [YR] = '20' and [BPPYER] = '20'
-- where [YR] = 20 and ([BPATYP] = 'OTHE' and [BPPYER] = '20')
-- order by [MSPIN]

-- 26 results
-- 22 results

-- select [MSPIN], [BPATYP], [BPPYER], [BPPCNB], [YR], [BPPISDC]
-- from [KCLCAS400].[S78114C0].[HTSUSR].[BPK001P2FL]
-- where [YR] = 20 and ([BPATYP] = 'SFD' or [BPATYP] = 'SFA' or [BPATYP] = 'MANH' or [BPATYP] = 'MHRP' or [BPATYP] = 'MHPS' or [BPATYP] = 'OTHE' and [BPPYER] = '20') and [MSPIN] = '8-00-14500-01-1905-00001'
-- order by [MSPIN]
-- 847 results
-- 789 results

-- select DISTINCT([MSPIN]), [BPATYP]
-- from [KCLCAS400].[S78114C0].[HTSUSR].[BPK001P2FL]
-- where [YR] = 20 and ([BPATYP] = 'SFD' or [BPATYP] = 'SFA' or [BPATYP] = 'MANH' or [BPATYP] = 'MHRP' or [BPATYP] = 'MHPS')
-- order by [MSPIN]
-- 835 results
-- 1-00-00400-01-0901-00001
-- select *
-- from gisadmin.DEMOPERMITSFINAL
-- where [Name] = '1-00-00400-01-0901-00001'

**Cell to create the All\_Permits Table from BPK001P2FL on AS400 Server.**

In [ ]:
SET ANSI_PADDING ON
GO

USE PROD
GO

-- CREATE Table [gisadmin].[MGOPERMIT](
--   [PARCELID] [varchar](24) NOT NULL
--   ,[StructureType] [varchar](4) NOT NULL
--   ,[PermitStatus] [varchar](2)  NOT NULL
--   ,[CODate] [varchar](24) NOT NULL
--   ,[AppDate] [varchar](24) NOT NULL
--   ,[IssueDate] [varchar](24) NOT NULL
--   ,[PermitYear] [varchar](24) NOT NULL
--   ,[PermitNumber] [varchar](24) NOT NULL
--   )
TRUNCATE Table [gisadmin].[All_Permits]

Insert into [gisadmin].[All_Permits] (
    [PARCELID],
    [Type],
    [Status],
    [DATE],
    [YEAR],
    [CODATE],    
    [APPDATE],
    [DESCRIPTION],
    [PermitDesc],
    [PermitNumber]
)
select [MSPIN], [BPATYP], [BPASTS], [BPPISDC], [YR], [BPPSTDC], [BPADATC], '', '', [BPPCNB]
from [KCLCAS400].[S78114C0].[HTSUSR].[BPK001P2FL] 
-- [KCC-SQL1].[Assess50StagedData_Kent].[dbo].[MGOPermitData]

# Random Queries

In [ ]:
(select a.PARCELID, a.AppType, a.PermitStatus
from prod.gisadmin.MGOPermits as a
where a.PARCELID = '1-00-01100-01-2604-00001')
UNION ALL
(select b.PARCELID, b.[TYPE] as [AppType], b.STATUS as [PermitStatus]
from prod.gisadmin.All_Permits as b
where b.PARCELID = '1-00-01100-01-2604-00001')

select *
from prod.gisadmin.MGOPermits as a
FULL JOIN prod.gisadmin.All_Permits as b
on a.PARCELID = b.PARCELID
where a.ParcelID is not null and b.PARCELID is not null 
order by a.PARCELID

In [ ]:
select *
from prod.gisadmin.All_Permits

In [ ]:
USE PROD
GO

INSERT INTO [KCC-SQL1].[Assess50StagedData_Kent].[dbo].[All_Permits] (
    [PARCELID],
    [ApplicationType],
    [Status],
    [ApplicationDate],
    [ProjectNumber],
    [IssueDate],
    [PermitYear],    
    [Description]
)
select f.*
FROM (select a.PARCELID, a.AppType, a.PermitStatus, FORMAT(CAST(LEFT(a.AppDate, 10) as date), 'd', 'en-US') as AppDate, a.PermitNumber, a.IssueDate, LEFT(a.PermitNumber, 4) as PermitYear, a.Designation
from prod.gisadmin.MGOPermits as a
UNION ALL
select g.PARCELID, g.AppType, g.PermitStatus, g.AppDate, g.PermitNumber, g.IssueDate, g.PermitYear, g.Designation
FROM (select b.PARCELID, b.[TYPE] as [AppType], b.STATUS as [PermitStatus], FORMAT(CAST(b.APPDATE as date), 'd', 'en-US') as AppDate, CONCAT(RIGHT(b.AppDate, 4),'-', b.PERMITNUMBER, '-', b.TYPE) as PermitNumber, 
FORMAT(CAST(b.Date as date), 'd', 'en-US') as IssueDate, RIGHT(b.APPDATE, 4) as PermitYear, p.PermitDesc as Designation
from prod.gisadmin.All_Permits as b
JOIN prod.gisadmin.All_Permits_Description as p
ON p.[Type] = b.[TYPE]
GROUP BY b.[TYPE], b.PARCELID, b.[STATUS], b.[DATE], p.PermitDesc, b.APPDATE, b.PERMITNUMBER
) as g) as f

In [ ]:
select d.*, p.PermitDesc
FROM (select a.PARCELID, a.AppType, a.PermitStatus, CAST(LEFT(a.AppDate, 10) as date) as AppDate, a.PermitNumber, a.IssueDate, LEFT(a.PermitNumber, 4) as PermitYear
from prod.gisadmin.MGOPermits as a
UNION ALL
select 
FROM (b.PARCELID, b.[TYPE] as [AppType], b.STATUS as [PermitStatus], b.APPDATE as AppDate, CONCAT(RIGHT(b.AppDate, 4),'-', b.PERMITNUMBER, '-', b.TYPE) as PermitNumber, 
CAST(b.Date as date) as IssueDate, RIGHT(b.APPDATE, 4) as PermitYear
from prod.gisadmin.All_Permits as f


) as d
JOIN prod.gisadmin.All_Permits_Description as p
ON p.[Type] = d.AppType
GROUP BY p.PermitDesc, d.PARCELID, d.AppType, d.AppDate, d.IssueDate, d.PermitStatus, d.PermitNumber, d.PermitYear
ORDER BY d.PARCELID

In [ ]:
select f.*
FROM (select a.PARCELID, a.AppType, a.PermitStatus, FORMAT(CAST(LEFT(a.AppDate, 10) as date), 'd', 'en-US') as AppDate, a.PermitNumber, a.IssueDate, LEFT(a.PermitNumber, 4) as PermitYear, a.Designation
from prod.gisadmin.MGOPermits as a
UNION ALL
select g.PARCELID, g.AppType, g.PermitStatus, g.AppDate, g.PermitNumber, g.IssueDate, g.PermitYear, g.Designation
FROM (select b.PARCELID, b.[TYPE] as [AppType], b.STATUS as [PermitStatus], FORMAT(CAST(b.APPDATE as date), 'd', 'en-US') as AppDate, CONCAT(RIGHT(b.AppDate, 4),'-', b.PERMITNUMBER, '-', b.TYPE) as PermitNumber, 
FORMAT(CAST(b.Date as date), 'd', 'en-US') as IssueDate, RIGHT(b.APPDATE, 4) as PermitYear, p.PermitDesc as Designation
from prod.gisadmin.All_Permits as b
JOIN prod.gisadmin.All_Permits_Description as p
ON p.[Type] = b.[TYPE]
GROUP BY b.[TYPE], b.PARCELID, b.[STATUS], b.[DATE], p.PermitDesc, b.APPDATE, b.PERMITNUMBER
) as g) as f

In [ ]:
select max(cast(right(a.APPDATE, 4) as int)) as PermitYear, a.PARCELID, a.PERMITNUMBER, a.[TYPE], a.[STATUS]
from prod.gisadmin.All_Permits as a
where ([TYPE] = 'SFD' or [TYPE] = 'SFA' or [TYPE] = 'SFD' or [TYPE] =  'MANH' or [TYPE]= 'MHRP' or [TYPE]= 'MHPS')
group by PARCELID, PERMITNUMBER, [TYPE], [STATUS]
order by PARCELID
-- format(cast(convert(varchar, max(PermitYear)) as date), 'd', 'en-US') as MaxYear

-- select a.[TYPE], a.PARCELID, a.[STATUS], a.[DATE], a.APPDATE, a.PERMITNUMBER, b.PermitDesc
-- from prod.gisadmin.All_Permits as a
-- full outer join prod.gisadmin.All_Permits_Description as b
-- on a.[TYPE] = b.[Type]
-- group by a.[TYPE], a.PARCELID, a.[STATUS], a.[DATE], a.APPDATE, a.PERMITNUMBER, b.PermitDesc

In [ ]:
select *
from prod.gisadmin.SUFFIX_PARCELS
where ParcelID like '1-17-01900-01-0102-2%'

In [ ]:
SELECT *
from prod.gisadmin.All_Permits_Description

select *
from prod.gisadmin.PERMIT_Description

In [1]:
select count(*)
from prod.gisadmin.MGOPermits

(1 row affected)

Total execution time: 00:00:00.005

(No column name)
722


In [ ]:
select *
from [KCLCAS400].[S78114C0].[HTSUSR].[BPK001P3]

In [ ]:
select Definition
from Engineering.dbo.GDB_ITEMS